In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [3]:
#look around what all do we have in our data set
#credits.head(1)['crew'].values

In [4]:
movies=movies.merge(credits,on='title')
#merging the 2 datasets on the basis of title
#movies.head(1)

In [5]:
#movies.info()

In [6]:
#movies['original_language'].value_counts()
# counts the movies of 1 language

#which columns will we keep out of this
#geners
#keywords
#title
language we do not need because max of the mivies are in english
#overview
#cast 
#crew


In [7]:
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [8]:
#movies.head()

In [9]:
movies.isnull().sum()
#3 columns have overview as null

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [10]:
movies.dropna(inplace=True)

In [11]:
#movies.isnull().sum()
# no null checked again

In [12]:
#movies.duplicated().sum()
#there are no duplicated columns 

In [13]:
movies.iloc[0].genres
#need to bring this in the format
#{'Action','Adventure','Fantasy','Scifi'}

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [14]:
#the current data we have in movies.genres is a String which has a list inside
#so we use an in built library in python
import ast
#ast.literal_eval()
#this takes in the string and gives out the list

In [15]:
# this is a helper function takes in a list of Genres and returns the keywords
def convert(obj):
    L=[] 
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [16]:
movies['genres'] = movies['genres'].apply(convert)
# for both keywords and genres the data and important keywords 
#are extracted

In [17]:
movies['keywords']=movies['keywords'].apply(convert)

In [18]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [19]:
#movies['cast'][0]
# the cast column contains a lot of useless data 
#we will remove only the top 3 actors.... and only the name if the case

In [20]:
def convert3(obj):
    L=[]
    counter =0
    for i in ast.literal_eval(obj):
        if counter!=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [21]:
movies['cast']=movies['cast'].apply(convert3)

In [22]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [23]:
def fetch_dir(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [24]:
movies['crew']=movies['crew'].apply(fetch_dir)

In [25]:
movies.head(1)
#now we will convert the overview into list

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [26]:
movies['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [27]:
movies['overview']=movies['overview'].apply(lambda x:x.split())
#converts the string into list

In [28]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [29]:
#now we want to remove spaces from between a name eg Sam Wha.. to Sammmo
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x]) 

In [30]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [31]:
new_df = movies[['movie_id','title','tags']]

In [32]:
#new_df.head(1)

In [33]:
#now the tags list is combined into qa string
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

/var/folders/d1/yxxtsrk50dz5gp63tp7p_6pr0000gn/T/ipykernel_24307/2113696568.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [34]:
#new_df.head()
new_df['tags'][0]
#convert all of this to lower case

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [35]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

/var/folders/d1/yxxtsrk50dz5gp63tp7p_6pr0000gn/T/ipykernel_24307/4224080999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [36]:
new_df['tags'][1]

"captain barbossa, long believed to be dead, has come back to life and is headed to the edge of the earth with will turner and elizabeth swann. but nothing is quite as it seems. adventure fantasy action ocean drugabuse exoticisland eastindiatradingcompany loveofone'slife traitor shipwreck strongwoman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger johnnydepp orlandobloom keiraknightley goreverbinski"

In [37]:
#For stemming the tags, reducing similar words to one words throughout the data
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [38]:
#helper functiont to perform stemming
def stem(text):
    y=[]
    
    for i in text.split():
        y.append(ps.stem(i))
       
    return  " ".join(y)

In [39]:
new_df['tags']=new_df['tags'].apply(stem)
new_df['tags'][0]
# we can see how romance becomes romac and similar stemming reduces words. 

/var/folders/d1/yxxtsrk50dz5gp63tp7p_6pr0000gn/T/ipykernel_24307/883448020.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

#ps.stem('loved') 
- out is 'love'
- just to show how .stem works

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
#we are combining the words and removing stop words from the string of tags 

In [41]:
#we will convert the sparse matrix returned by the Vectorizer to a numpy array
vectors=cv.fit_transform(new_df['tags']).toarray()
vectors.shape
# the size tells us 4806 movies and 5000 words

(4806, 5000)

In [42]:
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
similarity= cosine_similarity(vectors)
# array of arrays
similarity[0]
# each array has movie[ith] has 4086 distances from this to other movies

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

- used in the helper function below 
- #new_df[new_df['title']=='Avatar'].index[0]
- this line basically returns the index of a movie in our database
________________________________________

- used in the helper function below
#list(enumerate (similarity[0]))
- this list becomes a tuple 
- index position with distance
- now we wil use this to to create a sorted array
- to sort the tuple on the basis of 2nd element( the distance and not the index)
#This is done by using key=lambda x:x[1]

#sorted(similarity[0],reverse=True)
- by this way the array is sorted in reverse order so now we have lost the index of the movie
- so we need to store the indexes and then sort
- this is done using the code below, i have left it as code to see the working but has been used inside the fucntion


In [44]:
#sorted(list(enumerate(similarity[0])),reverse=True, key=lambda x:x[1])

- now to change the recommended numbers (movie indexes) to movie names
- we will change the movies_list to name
#new_df.iloc[1216].title

In [45]:
# create a function which can recommend the movies with shortest 5 distances
#we get an array of all the movie indexes
def recommend(movie):
    movie_index=new_df[new_df['title']== movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True, key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)


In [46]:
 recommend('Thor')

Thor: The Dark World
Clash of the Titans
After Earth
Ant-Man
Iron Man 2


In [47]:
import pickle
#this will make the files available in our select box in our website

In [48]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [49]:
pickle.dump(similarity,open('similarity.pkl','wb'))